## 03 - Modelling

## Optional - Colab Setup

In [ ]:
def upgrade_runtime_ram():
    meminfo = subprocess.getoutput('cat /proc/meminfo').split('\n')

    memory_info = {entry.split(':')[0]: int(entry.split(':')[1].replace(' kB','').strip()) for entry in meminfo}

    if memory_info['MemTotal'] > 17000000:
        return

    a = []
    while(1):
        a.append('1')

In [ ]:
def restart_runtime():
    os.kill(os.getpid(), 9)

In [ ]:
def setup_rapids():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    device_name = pynvml.nvmlDeviceGetName(handle)
    if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
        print("Wrong GPU - Restarting Runtime")
        restart_runtime()


    # clone RAPIDS AI rapidsai-csp-utils scripts repo
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git

    # install RAPIDS
    !bash rapidsai-csp-utils/colab/rapids-colab.sh 0.13


    # set necessary environment variables 
    dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
    sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
    sys.path

    # update pyarrow & modules 
    exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
def setup_conda():
    if not 'Miniconda3-4.5.4-Linux-x86_64.sh' in os.listdir():
        !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local

    if not ('EPFL-Capstone-Project' in os.listdir()) and (os.getcwd().split('/')[-1] != 'EPFL-Capstone-Project'):
        !git clone https://github.com/helmigsimon/EPFL-Capstone-Project  
    if 'EPFL-Capstone-Project' in os.listdir():
        os.chdir('EPFL-Capstone-Project')

    !conda env create -f environment.yml
    !conda activate exts-ml

In [ ]:
def setup_drive():
    #Mounting Google Drive
    global drive
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
try:
    import sys,os,subprocess
    
    upgrade_runtime_ram()
    setup_drive()

    #Setting up PyPi Packages
    !pip install geopandas sparse-dot-topn pdpipe category-encoders catboost
    import geopandas as gpd
    import sparse_dot_topn.sparse_dot_topn as ct
    import pdpipe as pdp
    import category_encoders

    #Setting up Conda Packages
    setup_conda()
    
    #Initializing NLTK
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    
    #Setting up RAPIDS AI
    import pynvml
    setup_rapids()
    
    from cuml import UMAP
    
except ModuleNotFoundError as e:
    print(e)
    print('Not in colab environment, continuing to run locally')
    from umap import UMAP

## Imports

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split,  StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
tqdm.pandas()

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.model_selection import KFold
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

In [3]:
from lib.transformers import *
from lib.pipelines import *
from lib.processing import save_to_pkl, load_from_pkl
from lib.visualization import GridSearchPlotter
from data.util.paths import DATA_PATH

In [4]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

In [5]:
try:
    from cuml import LinearRegression, RandomForestRegressor, Ridge
except ModuleNotFoundError:
    pass

In [6]:
from catboost import CatBoostRegressor

## Loading Data

In [7]:
df = load_from_pkl('final',DATA_PATH)

In [8]:
col_set = {
    'format': {
        'description': 'format_description_', 
        'name': 'format_name_', 
        'text': ('format_text_clean'),
        'quantity': ('format_quantity')
    },
    'geography': {
        'superregion': 'superregion_',
        'region': 'region_',
        'country': 'country_'
    },
    'timeperiod': {
        'period': 'period_',
        'era': 'era_'
    },
    'genre': 'genre_',
    'style': 'style_',
    'null': None,
    'indicator': lambda x: x.max() == 1 and x.min() == 0,
    'image_embedding': 'image_umap_'
}
column_store = ColumnStore()
column_store.fit(df,col_set)

## Preparing for Machine Learning

In [9]:
X_tr, X_te, y_tr, y_te = train_test_split(df.drop('market_value',axis=1),df.market_value)

In [10]:
df_sample = pd.concat([X_tr,y_tr],axis=1).sample(frac=0.1)
X_tr_sample = df_sample.drop('market_value',axis=1)
y_tr_sample = df_sample['market_value']

In [11]:
del df

## Setting up Dummy Regression for baseline

In [12]:
dummy_regressor = DummyRegressor(strategy='mean')

In [13]:
dummy_regressor.fit(X_tr,np.log(y_tr).values)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [14]:
dummy_pred = dummy_regressor.predict(X_te)

In [15]:
r2_score(y_te,np.exp(dummy_pred))

-0.078638794003417

In [16]:
mean_absolute_error(y_te,np.exp(dummy_pred))

10.44845462509688

## Linear Regression - Record Store

In [17]:
record_store_lr_transformer = ColumnTransformer(transformers=[
    ('year_encoder', OneHotEncoder(dtype=np.uint8,handle_unknown='ignore'), ['year'])
], remainder='passthrough')

In [29]:
record_store_lr_removal_columns = set([
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title',
])

In [19]:
record_store_lr_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','master_id','format_name'])),
    ('record_store_column_remover', ColumnRemover(record_store_lr_removal_columns)),
    ('preprocessing',record_store_lr_transformer),
    ('scaler', StandardScaler()),
    ('lr', GridSearchCV(
        LinearRegression(),
        param_grid={
            'normalize':[False,True]
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1,
    ))
])

NameError: name 'LinearRegression' is not defined

In [20]:
record_store_lr_pipe.fit(X_tr,np.log(y_tr).values)

NameError: name 'record_store_lr_pipe' is not defined

In [ ]:
record_store_lr_pipe['lr'].best_score_

In [ ]:
record_store_lr_pred = record_store_lr_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(record_store_lr_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(record_store_lr_pred))

## Linear Regression - Full Information

In [22]:
full_information_lr_transformer = ColumnTransformer(transformers=[
    ('units_for_sale_imputer', SimpleImputer(strategy='constant',fill_value=0),['units_for_sale']),
    ('average_rating_imputer', SimpleImputer(strategy='mean'),['average_rating']),
    ('year_encoder', OneHotEncoder(dtype=np.uint8), ['year'])
], remainder='passthrough')

In [32]:
full_information_columns = set([
    'units_for_sale','community_have','community_want','average_rating','rating_count'
])
full_information_lr_removal_columns = record_store_lr_removal_columns - full_information_columns

In [ ]:
full_information_lr_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','master_id','format_name'])),
    ('record_store_column_remover', ColumnRemover(full_information_lr_removal_columns)),
    ('preprocessing',full_information_lr_transformer),
    ('scaler', StandardScaler()),
    ('lr', GridSearchCV(
        LinearRegression(),
        param_grid={
            'normalize':[False,True]
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1,
    ))
])

In [20]:
full_information_lr_pipe.fit(X_tr,np.log(y_tr).values)

NameError: name 'record_store_lr_pipe' is not defined

In [ ]:
full_information_lr_pipe['lr'].best_score_

In [ ]:
full_information_lr_pred = record_store_lr_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(full_information_lr_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(full_information_lr_pred))

## Lasso Regression - Record Store

## Lasso Regression - Full Information

## Ridge Regression - Record Store

In [25]:
record_store_ridge_removal_columns, record_store_ridge_transformer, full_information_ridge_removal_columns, full_information_ridge_transformer = (
    record_store_lr_removal_columns,
    record_store_lr_transformer,
    full_information_lr_removal_columns,
    full_information_lr_transformer
)

In [26]:
record_store_ridge_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','format_name','master_id'])),
    ('record_store_column_remover', ColumnRemover(record_store_ridge_removal_columns)),
    ('preprocessing',record_store_ridge_transformer),
    ('scaler', StandardScaler()),
    ('ridge', GridSearchCV(
        Ridge(normalize=False,solver='eig'),
        param_grid={
            'alpha': np.linspace(1e-2,1e3,20),
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1,
        return_train_score=True
    ))
])

NameError: name 'Ridge' is not defined

In [ ]:
record_store_ridge_pipe.fit(X_tr,np.log(y_tr).values)

In [ ]:
plotter = GridSearchPlotter(record_store_ridge_pipe['ridge'])

In [ ]:
plotter.plot_validation_curve(parameter='alpha',ylim=(-0.75,-0.25))

In [ ]:
record_store_ridge_pipe['ridge'].best_score_

In [ ]:
record_store_ridge_pipe['ridge'].best_params_

In [ ]:
record_store_ridge_pred = record_store_ridge_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(record_store_ridge_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(record_store_ridge_pred))

## Ridge - Full Information

In [ ]:
full_information_ridge_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','format_name','master_id'])),
    ('record_store_column_remover', ColumnRemover(full_information_ridge_removal_columns)),
    ('preprocessing',full_information_ridge_transformer),
    ('scaler', StandardScaler()),
    ('ridge', GridSearchCV(
        Ridge(normalize=False,solver='eig'),
        param_grid={
            'alpha': np.linspace(1e-2,1e3,20),
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1,
        return_train_score=True
    ))
])

In [ ]:
full_information_ridge_pipe.fit(X_tr,np.log(y_tr).values)

In [ ]:
plotter = GridSearchPlotter(full_information_ridge_pipe['ridge'])

In [ ]:
plotter.plot_validation_curve(parameter='alpha',ylim=(-0.75,-0.25))

In [ ]:
full_information_ridge_pipe['ridge'].best_score_

In [ ]:
full_information_ridge_pipe['ridge'].best_params_

In [ ]:
full_information_ridge_pred = full_information_ridge_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(full_information_ridge_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(full_information_ridge_pred))

## Random Forest - Record Store

In [40]:
record_store_random_forest_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title'
]
record_store_random_forest_removal_columns.extend(
    column_store._indicator
)
record_store_random_forest_removal_columns = set(record_store_random_forest_removal_columns)

In [ ]:
record_store_random_forest_preprocessing_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','format_name','master_id','year'])),
    ('reduce_indicators', IndicatorReducer(indicators=column_store._indicator,algorithm=PCA,components=40)),
    ('record_store_column_remover', ColumnRemover(record_store_random_forest_removal_columns)),
    ('cast_to_32', FunctionTransformer(func=lambda x: x.values.astype(np.float32)))
])

random_forest = RandomForestRegressor()

record_store_random_forest_search_pipe = Pipeline([
    ('preprocessing',record_store_random_forest_preprocessing_pipe),
    ('grid_search', GridSearchCV(
        random_forest,
        cv=KFold(n_splits=5,shuffle=True),
        param_grid={
            'n_estimators': np.linspace(10,200,5).astype(int)
        },
        verbose=50,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        return_train_score=True
    ))
])

record_store_random_forest_pipe = Pipeline([
    ('preprocessing',random_forest_preprocessing_pipe),
    ('random_forest',random_forest)
])

In [ ]:
record_store_random_forest_search_pipe.fit(X_tr_sample,np.log(y_tr_sample))

In [ ]:
record_store_random_forest_plotter = GridSearchPlotter(random_forest_search_pipe.steps[-1][1])
record_store_random_forest_plotter.plot_validation_curve(parameter='n_estimators',ylim=(-1,0))

In [ ]:
record_store_random_forest_search_pipe.steps[-1][1].best_score_

In [ ]:
record_store_random_forest_search_pipe.steps[-1][1].best_params_

In [ ]:
record_store_random_forest_pipe.fit(X_tr,np.log(y_tr).values.astype(np.float32))

In [ ]:
record_store_random_forest_te_pred = random_forest_pipe.predict(X_te)

In [ ]:
record_store_r2_score(y_te,np.exp(random_forest_te_pred))

In [ ]:
record_store_mean_absolute_error(y_te,np.exp(random_forest_te_pred))

## Random Forest - Full Information

In [41]:
full_information_random_forest_removal_columns = record_store_random_forest_removal_columns - full_information_columns

In [ ]:
full_information_random_forest_transformer = ColumnTransformer(transformers=[
    ('units_for_sale_imputer', SimpleImputer(strategy='constant',fill_value=0),['units_for_sale']),
    ('average_rating_imputer', SimpleImputer(strategy='mean'),['average_rating']),
], remainder='passthrough')

In [ ]:
full_information_random_forest_preprocessing_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','format_name','master_id','year'])),
    ('reduce_indicators', IndicatorReducer(indicators=column_store._indicator,algorithm=PCA,components=40)),
    ('record_store_column_remover', ColumnRemover(full_information_random_forest_removal_columns)),
    ('cast_to_32', FunctionTransformer(func=lambda x: x.values.astype(np.float32)))
])

random_forest = RandomForestRegressor()

full_information_random_forest_search_pipe = Pipeline([
    ('preprocessing',full_information_random_forest_preprocessing_pipe),
    ('grid_search', GridSearchCV(
        random_forest,
        cv=KFold(n_splits=5,shuffle=True),
        param_grid={
            'n_estimators': np.linspace(10,200,5).astype(int)
        },
        verbose=50,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        return_train_score=True
    ))
])

full_information_random_forest_pipe = Pipeline([
    ('preprocessing',random_forest_preprocessing_pipe),
    ('random_forest',random_forest)
])

In [ ]:
full_information_random_forest_search_pipe.fit(X_tr_sample,np.log(y_tr_sample))

In [ ]:
full_information_random_forest_plotter = GridSearchPlotter(random_forest_search_pipe.steps[-1][1])
full_information_random_forest_plotter.plot_validation_curve(parameter='n_estimators',ylim=(-1,0))

In [ ]:
full_information_random_forest_search_pipe.steps[-1][1].best_score_

In [ ]:
full_information_random_forest_search_pipe.steps[-1][1].best_params_

In [ ]:
full_information_random_forest_pipe.fit(X_tr,np.log(y_tr).values.astype(np.float32))

In [ ]:
full_information_random_forest_te_pred = random_forest_pipe.predict(X_te)

In [ ]:
full_information_r2_score(y_te,np.exp(random_forest_te_pred))

In [ ]:
full_information_mean_absolute_error(y_te,np.exp(random_forest_te_pred))

## Catboost - Record Store

In [ ]:
record_store_catboost_removal_columns = set([
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title'
])

In [ ]:
record_store_catboost_processing_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('record_store_column_remover', ColumnRemover(record_store_catboost_removal_columns))
])

catboost = CatBoostRegressor(
    random_seed=0,
    cat_features=['year','format_text','master_id','format_name','artist','label','country'],
    boosting_type='Ordered',
    n_estimators=100,
    learning_rate=0.16681005,
    l2_leaf_reg=3,
    border_count=170,
    max_depth=10
)

record_store_catboost_grid_pipe = Pipeline([
    ('processing', catboost_processing_pipe),
    ('grid_search', GridSearchCV(
        catboost,
        param_grid={'max_depth': np.linspace(10,32,5).astype(int)},
        n_jobs=-1,
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=50,
        return_train_score=True)
    )
])

In [ ]:
record_store_catboost_grid_pipe.fit(X_tr_sample,np.log(y_tr_sample))

In [ ]:
record_store_catboost_plot = GridSearchPlotter(catboost_grid_pipe.steps[-1][1])
record_store_catboost_plot.plot_validation_curve(parameter='max_depth',ylim=(-1,0))

In [ ]:
record_store_catboost_grid_pipe.steps[-1][1].best_score_

In [ ]:
record_store_catboost_grid_pipe.steps[-1][1].best_params_

In [ ]:
record_store_catboost_pipe.fit(X_tr,np.log(y_tr))

In [ ]:
catboost_te_pred = catboost_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(record_store_catboost_te_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(record_store_catboost_te_pred))

## CatBoost - Full Information

In [ ]:
full_information_catboost_removal_columns = record_store_catboost_removal_columns - full_information_columns

In [ ]:
full_information_catboost_processing_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('record_store_column_remover', ColumnRemover(record_store_catboost_removal_columns))
])

full_information_catboost_grid_pipe = Pipeline([
    ('processing', catboost_processing_pipe),
    ('grid_search', GridSearchCV(
        catboost,
        param_grid={'max_depth': np.linspace(10,32,5).astype(int)},
        n_jobs=-1,
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=50,
        return_train_score=True)
    )
])

In [ ]:
full_information_catboost_grid_pipe.fit(X_tr_sample,np.log(y_tr_sample))

In [ ]:
full_information_catboost_plot = GridSearchPlotter(catboost_grid_pipe.steps[-1][1])
full_information_catboost_plot.plot_validation_curve(parameter='max_depth',ylim=(-1,0))

In [ ]:
full_information_catboost_grid_pipe.steps[-1][1].best_score_

In [ ]:
full_information_catboost_grid_pipe.steps[-1][1].best_params_

In [ ]:
full_information_catboost_pipe.fit(X_tr,np.log(y_tr))

In [ ]:
catboost_te_pred = catboost_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(cfull_information_atboost_te_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(full_information_catboost_te_pred))